### Creating the SP500 historical database
The goal of this notebook is to construct a SQL database with SP500 historical data from the past 5 years.<br>
We are going to use the IEX API to collect the data from the SP500 listed companies, make some data processing and save it to RDS SQL database on AWS.<br>
This approach was inspired on Kaggle S&P 500 stock data (https://www.kaggle.com/camnugent/sandp500) and part of the code was extracted from the script _getSandP.py_ found on the dataset folder of the challenge.

In [1]:
# import libraries
from datetime import datetime
from concurrent import futures

import pandas as pd
from pandas import DataFrame
import pandas_datareader.data as web
import os

To use IEX API (https://iexcloud.io/docs/api/) trough pandas_datareader, we need to sign up to the service and provide an API key. 


In [2]:
# Define my API key for IEX
os.environ["IEX_API_KEY"] = 'pk_c876f405f2e441cba4bd913e6d066a7f'

We are going to use the list of SP500 from DataHub (https://datahub.io/core/s-and-p-500-companies).
The file 

In [3]:
df_sp500 = pd.read_csv('constituents_csv.csv')
df_sp500.head()

,Symbol,Name,Sector
0,MMM,3M Company,Industrials
1,AOS,A.O. Smith Corp,Industrials
2,ABT,Abbott Laboratories,Health Care
3,ABBV,AbbVie Inc.,Health Care
4,ACN,Accenture plc,Information Technology


In [4]:
list_sp500 = df_sp500.Symbol.to_list()
len(list_sp500)

505

In [18]:
list_sp500[:10]

['MMM', 'AOS', 'ABT', 'ABBV', 'ACN', 'ATVI', 'AYI', 'ADBE', 'AAP', 'AMD']

There are a total of 505 companies listed on Standard & Poor's 500.<br>

Next, we are going to iterate through each company, get their stock data from the past 5 years and concatenate on a dataframe

In [6]:
end_time = datetime.now()
start_time = datetime(end_time.year - 5, end_time.month , end_time.day)
print(end_time)
print(start_time)

2019-11-01 13:51:30.587813
2014-11-01 00:00:00


In [7]:
stock_df = web.DataReader('AMZN','iex', start_time, end_time)

In [8]:
stock_df.T

date,2014-11-03,2014-11-04,2014-11-05,2014-11-06,2014-11-07,2014-11-10,2014-11-11,2014-11-12,2014-11-13,2014-11-14,...,2019-10-18,2019-10-21,2019-10-22,2019-10-23,2019-10-24,2019-10-25,2019-10-28,2019-10-29,2019-10-30,2019-10-31
open,306.24,303.99,303.01,296.69,297.82,300.00,305.94,310.75,312.00,317.12,...,1787.80,1769.66,1788.15,1761.30,1771.09,1697.55,1748.06,1774.81,1760.24,1775.99
high,308.01,304.25,304.00,297.00,299.99,305.80,312.38,312.58,317.89,332.88,...,1793.98,1785.88,1789.78,1770.05,1788.34,1764.21,1778.70,1777.00,1782.38,1792.00
low,303.24,300.17,294.16,292.29,296.29,300.00,303.65,308.79,311.51,316.40,...,1749.20,1765.00,1762.00,1742.00,1760.27,1695.00,1742.50,1755.81,1759.12,1771.48
close,305.72,302.81,296.52,296.64,299.86,305.11,312.01,311.51,316.48,327.82,...,1757.51,1785.66,1765.73,1762.17,1780.78,1761.33,1777.08,1762.71,1779.99,1776.66
volume,3244150.00,3427384.00,4674492.00,3758121.00,2647352.00,2809056.00,3807333.00,2182982.00,3686428.00,8877785.00,...,3366091.00,2224902.00,2234425.00,2190380.00,5204350.00,9626402.00,3708851.00,2276855.00,2449405.00,2781185.00


In [9]:
stock_df.T.loc[['open']]

date,2014-11-03,2014-11-04,2014-11-05,2014-11-06,2014-11-07,2014-11-10,2014-11-11,2014-11-12,2014-11-13,2014-11-14,...,2019-10-18,2019-10-21,2019-10-22,2019-10-23,2019-10-24,2019-10-25,2019-10-28,2019-10-29,2019-10-30,2019-10-31
open,306.24,303.99,303.01,296.69,297.82,300.0,305.94,310.75,312.0,317.12,...,1787.8,1769.66,1788.15,1761.3,1771.09,1697.55,1748.06,1774.81,1760.24,1775.99


In [10]:
stock_df.T.loc[['open','high']]

date,2014-11-03,2014-11-04,2014-11-05,2014-11-06,2014-11-07,2014-11-10,2014-11-11,2014-11-12,2014-11-13,2014-11-14,...,2019-10-18,2019-10-21,2019-10-22,2019-10-23,2019-10-24,2019-10-25,2019-10-28,2019-10-29,2019-10-30,2019-10-31
open,306.24,303.99,303.01,296.69,297.82,300.0,305.94,310.75,312.00,317.12,...,1787.80,1769.66,1788.15,1761.30,1771.09,1697.55,1748.06,1774.81,1760.24,1775.99
high,308.01,304.25,304.00,297.00,299.99,305.8,312.38,312.58,317.89,332.88,...,1793.98,1785.88,1789.78,1770.05,1788.34,1764.21,1778.70,1777.00,1782.38,1792.00


In [15]:
stock_df

,open,high,low,close,volume
date,,,,,
2014-11-03,306.24,308.01,303.24,305.72,3244150
2014-11-04,303.99,304.25,300.17,302.81,3427384
2014-11-05,303.01,304.00,294.16,296.52,4674492
2014-11-06,296.69,297.00,292.29,296.64,3758121
2014-11-07,297.82,299.99,296.29,299.86,2647352
...,...,...,...,...,...
2019-10-25,1697.55,1764.21,1695.00,1761.33,9626402
2019-10-28,1748.06,1778.70,1742.50,1777.08,3708851
2019-10-29,1774.81,1777.00,1755.81,1762.71,2276855


In [16]:
cols = ['open','high', 'low', 'close', 'volume']
stock_df = web.DataReader('AMZN','iex', start_time, end_time)
concaTdf = pd.concat([stock_df.T.loc['open'],
                      stock_df.T.loc['high'],
                      stock_df.T.loc['low'],
                      stock_df.T.loc['close'],
                      stock_df.T.loc['volume']], axis=1)
#open_series

Testing the IEX API fetch of multiple companies

In [30]:
stock_10_df = web.DataReader(list_sp500[:10],'iex', start_time, end_time)
stock_10_df

RemoteDataError: Unable to read URL: https://cloud.iexapis.com/stable/stock/market/batch?symbols=MMM%2CAOS%2CABT%2CABBV%2CACN%2CATVI%2CAYI%2CADBE%2CAAP%2CAMD&types=chart&range=max&token=pk_c876f405f2e441cba4bd913e6d066a7f
Response Text:
b'You have exceeded your allotted message quota. Please enable pay-as-you-go to regain access'

In [32]:
stock_10_df

date        Symbols  Attributes
2014-11-03  MMM      open               153.14
                     high               154.39
                     low                152.98
                     close              153.30
                     volume         2376427.00
                                      ...     
2019-10-31  AMD      open                32.98
                     high                34.34
                     low                 32.82
                     close               33.93
                     volume        67881623.00
Length: 62900, dtype: float64

The returned Dataframe is structured on a multilevel for the columns.
To create a specific row for each company day we are going to stack the columns.

In [29]:
stock_10_df) = stock_10_df.stack()


AttributeError: 'Series' object has no attribute 'stack'

In [ ]:
concaTdf = pd.concat([stock_df.T.loc['open'],
                      stock_df.T.loc['high'],
                      stock_df.T.loc['low'],
                      stock_df.T.loc['close'],
                      stock_df.T.loc['volume']], axis=1)

In [18]:
open_series = stock_df.T.loc['open'].loc['AMZN']
open_series

date
2014-10-31     305.15
2014-11-03     306.24
2014-11-04     303.99
2014-11-05     303.01
2014-11-06     296.69
               ...   
2019-10-24    1771.09
2019-10-25    1697.55
2019-10-28    1748.06
2019-10-29    1774.81
2019-10-30    1760.24
Name: AMZN, Length: 1258, dtype: float64

In [19]:
high_series = stock_df.T.loc['high'].loc['AMZN']
high_series

date
2014-10-31     305.72
2014-11-03     308.01
2014-11-04     304.25
2014-11-05     304.00
2014-11-06     297.00
               ...   
2019-10-24    1788.34
2019-10-25    1764.21
2019-10-28    1778.70
2019-10-29    1777.00
2019-10-30    1782.38
Name: AMZN, Length: 1258, dtype: float64

In [26]:
df_concat = pd.concat([open_series, high_series], axis=1)
df_concat.columns = ['open', 'high']
df_concat

,open,high
date,,
2014-10-31,305.15,305.72
2014-11-03,306.24,308.01
2014-11-04,303.99,304.25
2014-11-05,303.01,304.00
2014-11-06,296.69,297.00
...,...,...
2019-10-24,1771.09,1788.34
2019-10-25,1697.55,1764.21
2019-10-28,1748.06,1778.70


In [30]:
df_concat.open

date
2014-10-31     305.15
2014-11-03     306.24
2014-11-04     303.99
2014-11-05     303.01
2014-11-06     296.69
               ...   
2019-10-24    1771.09
2019-10-25    1697.55
2019-10-28    1748.06
2019-10-29    1774.81
2019-10-30    1760.24
Name: open, Length: 1258, dtype: float64

In [13]:
stock_df['open']['MSFT']

date
2014-10-31     46.94
2014-11-03     46.89
2014-11-04     47.30
2014-11-05     47.80
2014-11-06     47.86
               ...  
2019-10-24    139.39
2019-10-25    139.34
2019-10-28    144.40
2019-10-29    144.08
2019-10-30    143.52
Name: MSFT, Length: 1258, dtype: float64

In [7]:
stock_df.columns

MultiIndex([(  'open', 'MSFT'),
            (  'open', 'AMZN'),
            (  'high', 'MSFT'),
            (  'high', 'AMZN'),
            (   'low', 'MSFT'),
            (   'low', 'AMZN'),
            ( 'close', 'MSFT'),
            ( 'close', 'AMZN'),
            ('volume', 'MSFT'),
            ('volume', 'AMZN')],
           names=['Attributes', 'Symbols'])

In [8]:
stock_df.T

date                 2014-10-31   2014-11-03   2014-11-04   2014-11-05  \
Attributes Symbols                                                       
open       MSFT           46.94        46.89        47.30        47.80   
           AMZN          305.15       306.24       303.99       303.01   
high       MSFT           46.97        47.46        47.73        47.90   
           AMZN          305.72       308.01       304.25       304.00   
low        MSFT           46.48        46.73        47.25        47.26   
           AMZN          301.50       303.24       300.17       294.16   
close      MSFT           46.95        47.44        47.57        47.86   
           AMZN          305.46       305.72       302.81       296.52   
volume     MSFT     35849656.00  23130397.00  21530813.00  22449594.00   
           AMZN      4551795.00   3244150.00   3427384.00   4674492.00   

date                 2014-11-06   2014-11-07   2014-11-10   2014-11-11  \
Attributes Symbols                                                       
open       MSFT           47.86        48.92        48.65        48.85   
           AMZN          296.69       297.82       300.00       305.94   
high       MSFT           48.86        48.92        49.15        48.93   
           AMZN          297.00       299.99       305.80       312.38   
low        MSFT           47.79        48.29        48.55        48.65   
           AMZN          292.29       296.29       300.00       303.65   
close      MSFT           48.70        48.68        48.89        48.87   
           AMZN          296.64       299.86       305.11       312.01   
volume     MSFT     33037841.00  28000598.00  36370067.00  23445239.00   
           AMZN      3758121.00   2647352.00   2809056.00   3807333.00   

date                 2014-11-12   2014-11-13  ...   2019-10-17   2019-10-18  \
Attributes Symbols                            ...                             
open       MSFT           48.56        48.81  ...       140.95       139.76   
           AMZN          310.75       312.00  ...      1796.49      1787.80   
high       MSFT           48.92        49.65  ...       141.42       140.00   
           AMZN          312.58       317.89  ...      1798.85      1793.98   
low        MSFT           48.52        48.71  ...       139.02       136.56   
           AMZN          308.79       311.51  ...      1782.02      1749.20   
close      MSFT           48.78        49.61  ...       139.69       137.41   
           AMZN          311.51       316.48  ...      1787.48      1757.51   
volume     MSFT     22722123.00  26210433.00  ...  22102797.00  32284428.00   
           AMZN      2182982.00   3686428.00  ...   2713773.00   3366091.00   

date                 2019-10-21   2019-10-22   2019-10-23   2019-10-24  \
Attributes Symbols                                                       
open       MSFT          138.45       138.97       136.88       139.39   
           AMZN         1769.66      1788.15      1761.30      1771.09   
high       MSFT          138.50       140.01       137.45       140.42   
           AMZN         1785.88      1789.78      1770.05      1788.34   
low        MSFT          137.01       136.26       135.61       138.67   
           AMZN         1765.00      1762.00      1742.00      1760.27   
close      MSFT          138.43       136.37       137.24       139.94   
           AMZN         1785.66      1765.73      1762.17      1780.78   
volume     MSFT     20716039.00  27935270.00  31380309.00  37278399.00   
           AMZN      2224902.00   2234425.00   2190380.00   5204350.00   

date                 2019-10-25   2019-10-28   2019-10-29   2019-10-30  
Attributes Symbols                                                      
open       MSFT          139.34       144.40       144.08       143.52  
           AMZN         1697.55      1748.06      1774.81      1760.24  
high       MSFT          141.14       145.67       144.50       145.00  
           AMZN    

In [17]:
sp500_df = pd.DataFrame()

In [18]:
sp500_df

""


In [19]:
sp500_df = pd.concat([sp500_df, stock_df])

In [22]:
sp500_df = pd.concat([sp500_df, stock_df])

In [23]:
sp500_df

,open,high,low,close,volume
date,,,,,
2014-10-31,46.94,46.97,46.48,46.95,35849656
2014-11-03,46.89,47.46,46.73,47.44,23130397
2014-11-04,47.30,47.73,47.25,47.57,21530813
2014-11-05,47.80,47.90,47.26,47.86,22449594
2014-11-06,47.86,48.86,47.79,48.70,33037841
...,...,...,...,...,...
2019-10-24,1771.09,1788.34,1760.27,1780.78,5204350
2019-10-25,1697.55,1764.21,1695.00,1761.33,9626402
2019-10-28,1748.06,1778.70,1742.50,1777.08,3708851


In [56]:
### Get data for all SP500 companies
SP500_stock_df = web.DataReader(sp500_wikipedia[:100],'iex', '2019-10-30', '2019-10-31')

In [57]:
SP500_stock_df

Attributes    open                                                      \
Symbols        MMM    ABT   ABBV    ABMD     ACN   ATVI    ADBE    AMD   
date                                                                     
2019-10-30  168.23  83.32  78.51  184.07  185.15  55.25  273.25  32.93   

Attributes                 ...   volume                                     \
Symbols        AAP    AES  ...     CPRI      CAH     KMX      CCL      CAT   
date                       ...                                               
2019-10-30  163.99  17.08  ...  2299671  2807661  829045  4253720  3948056   

Attributes                                            
Symbols       CBOE    CBRE      CBS      CDW      CE  
date                                                  
2019-10-30  575191  846875  5622034  1035225  619110  

[1 rows x 500 columns]

In [41]:
# Define the start and end date
end_data = datetime.now()
start_time = datetime(end_data.year - 5, end_data.month , end_data.day)
print(end_data)
print(start_time)

2019-10-31 14:36:02.931490
2014-10-31 00:00:00


In [53]:
stock_df = web.DataReader('CELG','iex', '2019-10-30', end_data)

In [54]:
stock_df

,open,high,low,close,volume
date,,,,,
2019-10-30,106.47,107.33,106.31,107.17,3516614


### 2.1 Creating the RDS database using boto3
We are going to use boto3 to create PostreSQL RDS instance on AWS.<br>


In [34]:
import time
import boto3
import botocore

In [37]:
import boto3
iam = boto3.client('iam')

for user in iam.list_users()['Users']:
 print("User: {0}\nUserID: {1}\nARN: {2}\nCreatedOn: {3}\n".format(
 user['UserName'],
 user['UserId'],
 user['Arn'],
 user['CreateDate']
 )
 )

User: APIuser
UserID: AIDARJ4MCWBCGAZPRDIZB
ARN: arn:aws:iam::089951219780:user/APIuser
CreatedOn: 2019-05-21 19:53:01+00:00

User: EC2_user
UserID: AIDARJ4MCWBCHNIJEWAYW
ARN: arn:aws:iam::089951219780:user/EC2_user
CreatedOn: 2019-05-16 12:27:59+00:00

User: gabriel
UserID: AIDAJVEXIGTBJMKNVLVQM
ARN: arn:aws:iam::089951219780:user/gabriel
CreatedOn: 2019-01-23 23:09:35+00:00

User: gabriel-goncalves
UserID: AIDAIFXS6L6MDHCI7V2BS
ARN: arn:aws:iam::089951219780:user/gabriel-goncalves
CreatedOn: 2019-01-24 00:40:56+00:00

User: GabrielSGoncalves
UserID: AIDAITSKGIGW3H6GBFSCG
ARN: arn:aws:iam::089951219780:user/GabrielSGoncalves
CreatedOn: 2018-07-05 13:49:16+00:00

User: gustavo_fernandes
UserID: AIDARJ4MCWBCOYRJCL5XU
ARN: arn:aws:iam::089951219780:user/gustavo_fernandes
CreatedOn: 2019-08-21 17:33:30+00:00

User: jhonatan-teodoro
UserID: AIDARJ4MCWBCGFECRX7VA
ARN: arn:aws:iam::089951219780:user/jhonatan-teodoro
CreatedOn: 2019-05-21 19:06:26+00:00

User: lambda_etl_medium
UserID: AIDARJ

Create a new user with RDS and DynamoDB full access policy.

In [36]:
iam = boto3.client('iam')

# create a user
iam.create_user( UserName='lambda_etl_medium')

# attach a policy
iam.attach_user_policy(
 UserName = 'lambda_etl_medium', 
 PolicyArn='arn:aws:iam::aws:policy/AmazonRDSFullAccess'
)

{'ResponseMetadata': {'RequestId': '6ec4cecb-94be-4e6e-9c7e-91bba230981d',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '6ec4cecb-94be-4e6e-9c7e-91bba230981d',
   'content-type': 'text/xml',
   'content-length': '212',
   'date': 'Fri, 01 Nov 2019 17:15:59 GMT'},
  'RetryAttempts': 0}}

In [44]:
db_identifier = 'serverless_etl_medium'
rds = boto3.client('rds')

try:
    rds.create_db_instance(DBInstanceIdentifier=db_identifier,
                           AllocatedStorage=20,
                           DBName='etl_medium',
                           Engine='postgres',
                           # General purpose SSD
                           StorageType='gp2',
                           StorageEncrypted=True,
                           AutoMinorVersionUpgrade=True,
                           # Set this to true later?
                           MultiAZ=False,
                           MasterUsername='etl_medium',
                           MasterUserPassword='towardsdatascience',
                          # VpcSecurityGroupIds=['YOUR_SECURITY_GROUP_ID'],
                           DBInstanceClass='db.t2.micro'
                          # PubliclyAccessible=True
                           #Tags=[{'Key': 'MyTag', 'Value': 'Hawaii'}]
                          )
    print('Starting RDS instance with ID: %s' % db_identifier)
except botocore.exceptions.ClientError as e:
    print('DB instance %s exists already, continuing to poll ...' % db_identifier)

DB instance serverless_etl_medium exists already, continuing to poll ...


In [47]:
rds.create_db_instance(DBInstanceIdentifier=db_identifier,
                           AllocatedStorage=20,
                           DBName='etl-medium',
                           Engine='postgres',
                           # General purpose SSD
                           StorageType='gp2',
                           StorageEncrypted=True,
                           AutoMinorVersionUpgrade=True,
                           # Set this to true later?
                           MultiAZ=False,
                           MasterUsername='etl-medium',
                           MasterUserPassword='towardsdatascience',
                           DBInstanceClass='db.t2.micro')
                          # VpcSecurityGroupIds=['YOUR_SECURITY_GROUP_ID'],
                           

ClientError: An error occurred (InvalidParameterValue) when calling the CreateDBInstance operation: The parameter DBInstanceIdentifier is not a valid identifier. Identifiers must begin with a letter; must contain only ASCII letters, digits, and hyphens; and must not end with a hyphen or contain two consecutive hyphens.

In [48]:
rds.describe_db_instances()

{'DBInstances': [],
 'ResponseMetadata': {'RequestId': '2722e356-e061-45ad-adcf-da4ba0d282cd',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '2722e356-e061-45ad-adcf-da4ba0d282cd',
   'content-type': 'text/xml',
   'content-length': '297',
   'date': 'Fri, 01 Nov 2019 17:36:58 GMT'},
  'RetryAttempts': 0}}